In [99]:
import numpy as np
from statsmodels.stats.contingency_tables import mcnemar
import mlflow

import ast

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.inspection import permutation_importance
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from graph_reinforcement_learning_using_blockchain_data import config
from graph_reinforcement_learning_using_blockchain_data.modeling import random_forest

config.load_dotenv()

True

## RF with GNN embeddings

In [100]:
df_features = pd.read_csv(
    config.PROCESSED_DATA_DIR / "flashbots" / "Q2_2023" / "features_edges_without_0logs.csv"
)
df_embeddings = pd.read_csv(config.FLASHBOTS_Q2_DATA_DIR / "embeddings_128.csv")

In [101]:
def parse_embedding(embedding_str):
    embedding_str = embedding_str.replace("\n", " ")
    embedding_array = np.fromstring(embedding_str.strip("[]"), sep=" ")
    return embedding_array


df_embeddings['embeddings'] = df_embeddings['embeddings'].apply(parse_embedding)

In [102]:
embedding_list = df_embeddings['embeddings'].tolist()
embedding_dim = embedding_list[0].shape[0]  # e.g. 128

embeddings_expanded = pd.DataFrame(
    embedding_list, 
    columns=[f"emb_{i}" for i in range(embedding_dim)]
)
embeddings_expanded['transactionHash'] = df_embeddings['transactionHash'].values

In [103]:
df_merged = df_features.merge(embeddings_expanded, how="inner", on="transactionHash")

In [104]:
df = df_merged.drop(columns=["transactionHash", "from", "to", "blockNumber", "transactionIndex"])
df.head()

,gasUsed,cumulativeGasUsed,effectiveGasPrice,status,fee,num_logs,dummy_0xe9149e1b,dummy_0x8c5be1e5,dummy_0x7fcf532c,dummy_0xe1fffcc4,...,emb_118,emb_119,emb_120,emb_121,emb_122,emb_123,emb_124,emb_125,emb_126,emb_127
0,153529,10750253,19150328915,1,2940130847991035,3,0,1,0,0,...,0.703639,-0.676612,-0.518422,-0.524117,-0.732895,0.692418,-0.523640,-0.374066,-0.332854,-0.970089
1,123497,229900,139502435999,1,17228132338568503,7,0,0,0,0,...,-0.138172,0.087557,0.063972,0.171073,0.106778,-0.096407,0.173654,0.108045,0.091411,0.204592
2,46817,14885776,18825121161,1,881335697394537,1,0,0,0,0,...,0.703639,-0.676612,-0.518422,-0.524117,-0.732895,0.692418,-0.523640,-0.374066,-0.332854,-0.970089
3,32215,3160909,19660583885,1,633365709855275,1,0,0,0,0,...,0.703639,-0.676612,-0.518422,-0.524117,-0.732895,0.692418,-0.523640,-0.374066,-0.332854,-0.970089
4,153886,9279317,45282851253,1,6968396847919158,5,0,0,0,1,...,1.046832,-1.244183,-0.525696,-0.923329,-1.299295,0.577549,-0.721241,-1.122780,-1.086308,-0.922723


In [105]:
X = df.drop(columns=["label"])
# X = np.hstack([other_features, embedding_matrix])
y = df["label"]

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [107]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

model_uri_gnn = 'runs:/7b43596526c04f28b691185b808233fb/best_estimator'
model_uri_non_gnn = 'runs:/346e595d41db4b8ba68353a24e369c09/best_estimator'

model_gnn = mlflow.sklearn.load_model(model_uri_gnn)
model_non_gnn = mlflow.sklearn.load_model(model_uri_non_gnn)

In [108]:
gnn_pred = model_gnn.predict(X_test)

## RF with standard features

In [109]:
df_features = pd.read_csv(
    config.PROCESSED_DATA_DIR / "flashbots" / "Q2_2023" / "features_edges_without_0logs.csv"
)

In [110]:
df = df_features.drop(columns=["transactionHash", "from", "to", "blockNumber", "transactionIndex"])

In [111]:
X = df.drop(columns=["label"])
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [112]:
rf_pred = model_non_gnn.predict(X_test)

## McNemar's Test

In [113]:
def mcnemar_test(y_true, y_pred_a, y_pred_b):
    """
    Computes the McNemar contingency table and performs the test.
    
    Parameters:
        y_true: array-like, true labels.
        y_pred_a: array-like, predictions from classifier A.
        y_pred_b: array-like, predictions from classifier B.
    
    Returns:
        table: 2x2 contingency table.
        stat: McNemar's test statistic.
        pvalue: p-value for the test.
    """
    y_true = np.array(y_true)
    y_pred_a = np.array(y_pred_a)
    y_pred_b = np.array(y_pred_b)

    # Compute elements of the contingency table:
    # n00: both classifiers correct
    # n01: classifier A correct, classifier B wrong
    # n10: classifier A wrong, classifier B correct
    # n11: both classifiers wrong
    n00 = np.sum((y_pred_a == y_true) & (y_pred_b == y_true))
    n01 = np.sum((y_pred_a == y_true) & (y_pred_b != y_true))
    n10 = np.sum((y_pred_a != y_true) & (y_pred_b == y_true))
    n11 = np.sum((y_pred_a != y_true) & (y_pred_b != y_true))
    
    table = [[n00, n01],
             [n10, n11]]
    
    # Use exact binomial test for small sample sizes (exact=True).
    result = mcnemar(table, exact=True)
    return table, result.statistic, result.pvalue

The code compares the predictions for each test instance to build a 2×2 table:
- n00: Number of instances where both classifiers are correct.
- n01: Instances where classifier A is correct and classifier B is wrong.
- n10: Instances where classifier A is wrong and classifier B is correct.
- n11: Instances where both classifiers are wrong.

In [114]:
table, stat, pvalue = mcnemar_test(y_test, rf_pred, gnn_pred)
print("Contingency Table:", table)
print("McNemar's Test Statistic:", stat)
print("p-value:", pvalue)

Contingency Table: [[36264, 19], [277, 267]]
McNemar's Test Statistic: 19.0
p-value: 6.917112456951167e-60
